In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [12]:
!pip install langchain==0.3.10
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install jq==1.8.0
!pip install pymupdf==1.25.1

  Using cached langchain-0.3.10-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain-0.3.10-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.11 requires langchain<0.4.0,>=0.3.11, but you have langchain 0.3.10 which is incompatible.


  Using cached langchain-0.3.27-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.72-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl.metadata (1.9 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_core-0.3.71-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.70-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.69-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.68-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.67-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.3.66-py3-none-any.whl.metadata (5.8 kB)
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 94.2 MB/s eta 0:00:00


In [7]:
!pip install langchain-chroma==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.9 MB/s eta 0

In [5]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


In [6]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [7]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [8]:
import os
file_path = '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/'

In [39]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

#Contextual Chunking using LLm

In [40]:
# create chunk context generation chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser


def generate_chunk_context(document, chunk):

    chunk_process_prompt = """You are an AI assistant specializing in research paper analysis.
                            Your task is to provide brief, relevant context for a chunk of text
                            based on the following research paper.

                            Here is the research paper:
                            <paper>
                            {paper}
                            </paper>

                            Here is the chunk we want to situate within the whole document:
                            <chunk>
                            {chunk}
                            </chunk>

                            Provide a concise context (3-4 sentences max) for this chunk,
                            considering the following guidelines:

                            - Give a short succinct context to situate this chunk within the overall document
                            for the purposes of improving search retrieval of the chunk.
                            - Answer only with the succinct context and nothing else.
                            - Context should be mentioned like 'Focuses on ....'
                            do not mention 'this chunk or section focuses on...'

                            Context:
                        """

    prompt_template = ChatPromptTemplate.from_template(chunk_process_prompt)

    agentic_chunk_chain = (prompt_template
                                |
                            chatgpt
                                |
                            StrOutputParser())

    context = agentic_chunk_chain.invoke({'paper': document, 'chunk': chunk})

    return context

In [44]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import uuid

def create_contextual_chunks(file_path, chunk_size=3500, chunk_overlap=0):

    print('Loading pages:', file_path)
    loader = PyMuPDFLoader(file_path)
    doc_pages = loader.load()

    print('Chunking pages:', file_path)
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                              chunk_overlap=chunk_overlap)
    doc_chunks = splitter.split_documents(doc_pages)

    print('Generating contextual chunks:', file_path)
    original_doc = '\n'.join([doc.page_content for doc in doc_chunks])
    contextual_chunks = []
    for chunk in doc_chunks:
        chunk_content = chunk.page_content
        chunk_metadata = chunk.metadata
        chunk_metadata_upd = {
            'id': str(uuid.uuid4()),
            'page': chunk_metadata['page'],
            'source': chunk_metadata['source'],
            'title': chunk_metadata['source'].split('/')[-1]
        }
        context = generate_chunk_context(original_doc, chunk_content)
        contextual_chunks.append(Document(page_content=context+'\n'+chunk_content,
                                          metadata=chunk_metadata_upd))
    print('Finished processing:', file_path)
    print()
    return contextual_chunks

In [47]:
from glob import glob

pdf_files = glob(file_path + '*.pdf')
pdf_files

['/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf',
 '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.11401v4.pdf',
 '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.14165v4.pdf']

In [45]:
paper_docs = []
for fp in pdf_files:
    paper_docs.extend(create_contextual_chunks(file_path=fp, chunk_size=3500))

Loading pages: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf
Chunking pages: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf
Generating contextual chunks: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf
Finished processing: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf

Loading pages: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.11401v4.pdf
Chunking pages: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.11401v4.pdf
Generating contextual chunks: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.11401v4.pdf
Finished processing: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.11401v4.pdf

Loading pages: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.14165v4.pdf
Chunking pages: /content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.14165v4.pdf
Generating contextual chunks: /co

In [46]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid

def create_contextual_chunks(file_path, chunk_size=3500, chunk_overlap=0):

    print('Loading pages:', file_path)
    loader = PyMuPDFLoader(file_path)
    doc_pages = loader.load()

    print('Chunking pages:', file_path)
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                              chunk_overlap=chunk_overlap)
    doc_chunks = splitter.split_documents(doc_pages)

    print('Generating contextual chunks:', file_path)
    original_doc = '\n'.join([doc.page_content for doc in doc_chunks])
    contextual_chunks = []
    for chunk in doc_chunks:
        chunk_content = chunk.page_content
        chunk_metadata = chunk.metadata
        chunk_metadata_upd = {
            'id': str(uuid.uuid4()),
            'page': chunk_metadata['page'],
            'source': chunk_metadata['source'],
            'title': chunk_metadata['source'].split('/')[-1]
        }
        context = generate_chunk_context(original_doc, chunk_content)
        contextual_chunks.append(Document(page_content=context+'\n'+chunk_content,
                                          metadata=chunk_metadata_upd))
    print('Finished processing:', file_path)
    print()
    return contextual_chunks

In [14]:
len(paper_docs)

158

In [48]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=paper_docs,
                                  collection_name='my_context_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./my_context_db")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [49]:
# load from disk
chroma_db = Chroma(persist_directory="./my_context_db",
                   collection_name='my_context_db',
                   embedding_function=openai_embed_model)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [50]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

In [51]:
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content[:1000]))
        print()

In [52]:
query = "what is LLM?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Metadata: {'id': '3629b89c-9f1b-4526-a943-7004d3ae3d8f', 'page': 70, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.14165v4.pdf', 'title': '2005.14165v4.pdf'}
Content Brief:


Focuses on the references to various research papers that contribute to the development and understanding of language models, particularly in the context of self-supervised learning, adversarial training, and multi-task learning. These citations support the broader discussions on model architecture, training methodologies, and performance evaluations throughout the document.
[LCG+19] Zhenzhong Lan, Mingda Chen, Sebastian Goodman, Kevin Gimpel, Piyush Sharma, and Radu Sori-
cut. ALBERT: A lite BERT for self-supervised learning of language representations. arXiv preprint
arXiv:1909.11942, 2019.
[LCH+20] Xiaodong Liu, Hao Cheng, Pengcheng He, Weizhu Chen, Yu Wang, Hoifung Poon, and Jianfeng Gao.
Adversarial training for large neural language models. arXiv preprint arXiv:2004.08994, 2020.
[LDL19] Zhongyang Li, Xiao Ding, and Ting Liu. Story ending prediction by transferable bert. arXiv preprint
arXiv:1905.07504, 2019.
[LDM12] Hector Levesque, Ernest Davis, and Leora Morgenstern. The Winogr


Metadata: {'id': 'c0e96737-ca11-45c9-b21c-f63ba10694b5', 'page': 3, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.14165v4.pdf', 'title': '2005.14165v4.pdf'}
Content Brief:


Focuses on the concept of meta-learning in language models, particularly in the context of few-shot learning where demonstrations are provided at inference time. It addresses the ambiguity regarding whether models learn new tasks from scratch or recognize previously seen patterns, highlighting the significance of this distinction in understanding the model's capabilities. The discussion emphasizes the inner-outer loop structure of meta-learning as a framework for these processes.
demonstrations are provided at inference time. These terms are intended to remain agnostic on the question of whether the model
learns new tasks from scratch at inference time or simply recognizes patterns seen during training – this is an important issue which
we discuss later in the paper, but “meta-learning” is intended to encompass both possibilities, and simply describes the inner-outer
loop structure.
4

...

Focuses on the analysis of attention mechanisms in the Transformer model, specifically illustrating how certain attention heads are involved in resolving anaphora within sentences. The example provided demonstrates the model's ability to capture dependencies and relationships between words, highlighting the interpretability of attention distributions in understanding sentence structure.
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
The
Law
will
never
be
perfect
,
but
its
application
should
be
just
-
this
is
what
we
are
missing
,
in
my
opinion
.
<EOS>
<pad>
Figure 4: Two attention heads, also in layer 5 of 6, apparently involved in anaphora resolution. Top:



Metadata: {'id': '1c207495-f6bd-4e0f-84ed-ffd0c93a8d6e', 'page': 69, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.14165v4.pdf', 'title': '2005.14165v4.pdf'}
Content Brief:


Focuses on references to prior research and methodologies relevant to natural language processing, including sequence-level knowledge distillation and cross-lingual language model pretraining. These citations support the development and evaluation of language models discussed throughout the paper, particularly in relation to improving model performance and efficiency.
Linguistics, 2019.
[KR16] Yoon Kim and Alexander M. Rush. Sequence-level knowledge distillation. Arxiv, 2016.
[LB02] Edward Loper and Steven Bird. Nltk: The natural language toolkit, 2002.
[LC19] Guillaume Lample and Alexis Conneau. Cross-lingual language model pretraining. arXiv preprint
arXiv:1901.07291, 2019.
70


Metadata: {'id': 'c33ea655-1d46-48ec-8949-142978770721', 'page': 17, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.11401v4.pdf', 'title': '2005.11401v4.pdf'}
Content Brief:


Focuses on the exploration of null document mechanisms in the RAG model, discussing various methods for predicting logits for a null document and their lack of performance improvement. It also highlights observations from the Open MS-MARCO task regarding the model's tendency to retrieve a consistent set of documents for less retrieval-beneficial questions, suggesting that null document mechanisms may not be essential for the RAG architecture.
logit by learning (i) a document embedding for the null document, (ii) a static learnt bias term, or
(iii) a neural network to predict the logit. We did not ﬁnd that these improved performance, so in
the interests of simplicity, we omit them. For Open MS-MARCO, where useful retrieved documents
cannot always be retrieved, we observe that the model learns to always retrieve a particular set of
documents for questions that are less likely to beneﬁt from retrieval, suggesting that null document
mechanisms may not be necessary for RAG.
G
Parameters
Our

In [53]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [58]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

src_rag_response_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs)),
        "question": itemgetter("question")
    }
        |
    rag_prompt_template
        |
    chatgpt
        |
    StrOutputParser()
)

rag_chain_w_sources = (
    {
        "context": similarity_retriever,
        "question": RunnablePassthrough()
    }
        |
    RunnablePassthrough.assign(response=src_rag_response_chain)
)

In [59]:
from IPython.display import display, Markdown

def display_results(result_obj):
    print('Query:')
    display(Markdown(result_obj['question']))
    print()
    print('Response:')
    display(Markdown(result_obj['response']))
    print('='*50)
    print('Sources:')
    for source in result_obj['context']:
        print('Metadata:', source.metadata)
        print('Content Brief:')
        display(Markdown(source.page_content))
        print()


In [65]:
query = "What are the main components of RAG model, and how do they interact?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What are the main components of RAG model, and how do they interact?


Response:


The main components of the Retrieval-Augmented Generation (RAG) model are the **retriever** and the **generator**. These components interact in a structured manner to enhance the model's ability to generate factual and contextually relevant responses.

### 1. Retriever
- **Function**: The retriever, denoted as \( p_{\eta}(z|x) \), is responsible for retrieving relevant text documents based on a given input query \( x \). It returns a distribution over the top-K text passages that are most relevant to the query.
- **Architecture**: The retriever is based on a Dense Passage Retrieval (DPR) architecture, which utilizes a bi-encoder setup. It generates dense representations for both documents and queries using BERT, allowing for efficient retrieval of relevant documents.
- **Process**: The retriever calculates the top-K documents with the highest prior probability \( p_{\eta}(z|x) \) through a Maximum Inner Product Search (MIPS) approach.

### 2. Generator
- **Function**: The generator, denoted as \( p_{\theta}(y_i|x, z, y_{1:i-1}) \), takes the input query \( x \) and the retrieved documents \( z \) to generate the target output sequence \( y \). It produces the next token in the sequence based on the previous tokens and the context provided by the retrieved documents.
- **Architecture**: The generator is implemented using BART, a pre-trained sequence-to-sequence transformer model. It combines the input query and the retrieved content by concatenating them before generating the output.
- **Process**: The generator can operate in two modes:
  - **RAG-Sequence**: Uses the same retrieved document to generate the entire output sequence.
  - **RAG-Token**: Allows for the selection of different documents for each token in the output sequence, providing more flexibility and diversity in the generated responses.

### Interaction Between Components
- The interaction between the retriever and generator is crucial for the RAG model's performance. The retriever first identifies and retrieves relevant documents based on the input query. These documents serve as additional context for the generator, which then produces a response that is informed by both the input query and the retrieved documents.
- This retrieval-augmented generation process allows RAG models to generate responses that are more factual, specific, and diverse compared to traditional models like BART, particularly in tasks requiring fact verification and open-domain question answering.

### Summary
In summary, the RAG model integrates a retriever and a generator to enhance its ability to produce high-quality, contextually relevant outputs. The retriever fetches pertinent documents, while the generator synthesizes this information to generate coherent and informative responses.

Sources:
Metadata: {'id': 'a52a5b98-b18f-4881-b701-906f0f7a54ee', 'page': 1, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.11401v4.pdf', 'title': '2005.11401v4.pdf'}
Content Brief:


...

Focuses on the implementation details, human evaluation methods, and training setup for the Retrieval-Augmented Generation (RAG) models used in knowledge-intensive NLP tasks. It outlines the specific configurations for open-domain question answering and the evaluation process for assessing model performance. Additionally, it discusses the technical aspects of training the models and the resources utilized.
Appendices for Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
A
Implementation Details
For Open-domain QA we report test numbers using 15 retrieved documents for RAG-Token models.
For RAG-Sequence models, we report test results using 50 retrieved documents, and we use the
Thorough Decoding approach since answers are generally short. We use greedy decoding for QA as
we did not ﬁnd beam search improved results. For Open-MSMarco and Jeopardy question generation,
we report test numbers using ten retrieved documents for both RAG-Token and RAG-Sequence,
and we also train a BART-large model as a baseline. We use a beam size of four, and use the Fast
Decoding approach for RAG-Sequence models, as Thorough Decoding did not improve performance.
B
Human Evaluation
Figure 4: Annotation interface for human evaluation of factuality. A pop-out for detailed instructions
and a worked example appear when clicking "view tool guide".
Figure 4 shows the user interface for human evaluation. To avoid any biases for screen position,
which model corresponded to sentence A and sentence B was randomly selected for each example.
Annotators were encouraged to research the topic using the internet, and were given detailed instruc-
tions and worked examples in a full instructions tab. We included some gold sentences in order to
assess the accuracy of the annotators. Two annotators did not perform well on these examples and
their annotations were removed from the results.
C
Training setup Details
We train all RAG models and BART baselines using Fairseq [45].2 We train with mixed precision
ﬂoating point arithmetic [40], distributing training across 8, 32GB NVIDIA V100 GPUs, though
training and inference can be run on one GPU. We ﬁnd that doing Maximum Inner Product Search
with FAISS is sufﬁciently fast on CPU, so we store document index vectors on CPU, requiring ∼100
GB of CPU memory for all of Wikipedia. After submission, We have ported our code to HuggingFace
Transformers [66]3, which achieves equivalent performance to the previous version but is a cleaner
and easier to use implementation. This version is also open-sourced. We also compress the document
index using FAISS’s compression tools, reducing the CPU memory requirement to 36GB. Scripts to
run experiments with RAG can be found at https://github.com/huggingface/transformers/
blob/master/examples/rag/README.md and an interactive demo of a RAG model can be found
at https://huggingface.co/rag/
2https://github.com/pytorch/fairseq
3https://github.com/huggingface/transformers
17


Metadata: {'id': 'dc233751-5760-4a63-a92d-a49c814413a1', 'page': 2, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.11401v4.pdf', 'title': '2005.11401v4.pdf'}
Content Brief:


Focuses on the description of the RAG-Sequence and RAG-Token models, detailing how they generate text by utilizing retrieved documents as latent variables. It explains the marginalization process for generating sequences and introduces the components involved, including the retriever based on DPR and the generator using BART. Additionally, it outlines the training methodology for these models.
by θ that generates a current token based on a context of the previous i −1 tokens y1:i−1, the original
input x and a retrieved passage z.
To train the retriever and generator end-to-end, we treat the retrieved document as a latent variable.
We propose two models that marginalize over the latent documents in different ways to produce a
distribution over generated text. In one approach, RAG-Sequence, the model uses the same document
to predict each target token. The second approach, RAG-Token, can predict each target token based
on a different document. In the following, we formally introduce both models and then describe the
pη and pθ components, as well as the training and decoding procedure.
2.1
Models
RAG-Sequence Model
The RAG-Sequence model uses the same retrieved document to generate
the complete sequence. Technically, it treats the retrieved document as a single latent variable that
is marginalized to get the seq2seq probability p(y|x) via a top-K approximation. Concretely, the
top K documents are retrieved using the retriever, and the generator produces the output sequence
probability for each document, which are then marginalized,
pRAG-Sequence(y|x) ≈
X
z∈top-k(p(·|x))
pη(z|x)pθ(y|x, z) =
X
z∈top-k(p(·|x))
pη(z|x)
N
Y
i
pθ(yi|x, z, y1:i−1)
RAG-Token Model
In the RAG-Token model we can draw a different latent document for each
target token and marginalize accordingly. This allows the generator to choose content from several
documents when producing an answer. Concretely, the top K documents are retrieved using the
retriever, and then the generator produces a distribution for the next output token for each document,
before marginalizing, and repeating the process with the following output token, Formally, we deﬁne:
pRAG-Token(y|x) ≈
N
Y
i
X
z∈top-k(p(·|x))
pη(z|x)pθ(yi|x, z, y1:i−1)
Finally, we note that RAG can be used for sequence classiﬁcation tasks by considering the target class
as a target sequence of length one, in which case RAG-Sequence and RAG-Token are equivalent.
2.2
Retriever: DPR
The retrieval component pη(z|x) is based on DPR [26]. DPR follows a bi-encoder architecture:
pη(z|x) ∝exp
 d(z)⊤q(x)

d(z) = BERTd(z), q(x) = BERTq(x)
where d(z) is a dense representation of a document produced by a BERTBASE document encoder [8],
and q(x) a query representation produced by a query encoder, also based on BERTBASE. Calculating
top-k(pη(·|x)), the list of k documents z with highest prior probability pη(z|x), is a Maximum Inner
Product Search (MIPS) problem, which can be approximately solved in sub-linear time [23]. We use
a pre-trained bi-encoder from DPR to initialize our retriever and to build the document index. This
retriever was trained to retrieve documents which contain answers to TriviaQA [24] questions and
Natural Questions [29]. We refer to the document index as the non-parametric memory.
2.3
Generator: BART
The generator component pθ(yi|x, z, y1:i−1) could be modelled using any encoder-decoder. We use
BART-large [32], a pre-trained seq2seq transformer [58] with 400M parameters. To combine the input
x with the retrieved content z when generating from BART, we simply concatenate them. BART was
pre-trained using a denoising objective and a variety of different noising functions. It has obtained
state-of-the-art results on a diverse set of generation tasks and outperforms comparably-sized T5
models [32]. We refer to the BART generator parameters θ as the parametric memory henceforth.
2.4
Training


Metadata: {'id': 'e8db79fe-53fb-4731-8a2b-30c060d96b9e', 'page': 6, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.11401v4.pdf', 'title': '2005.11401v4.pdf'}
Content Brief:


Focuses on the evaluation of RAG's retrieval mechanism, comparing its performance with a fixed BM25 system for the FEVER task, highlighting the effectiveness of differentiable retrieval for various tasks, particularly Open-Domain QA. It also discusses the advantage of non-parametric memory models like RAG in updating knowledge at test time, contrasting this with the limitations of parametric-only models.
RAG’s retriever with a ﬁxed BM25 system, and use BM25 retrieval scores as logits when calculating
p(z|x). Table 6 shows the results. For FEVER, BM25 performs best, perhaps since FEVER claims are
heavily entity-centric and thus well-suited for word overlap-based retrieval. Differentiable retrieval
improves results on all other tasks, especially for Open-Domain QA, where it is crucial.
Index hot-swapping
An advantage of non-parametric memory models like RAG is that knowledge
can be easily updated at test time. Parametric-only models like T5 or BART need further training to
update their behavior as the world changes. To demonstrate, we build an index using the DrQA [5]
Wikipedia dump from December 2016 and compare outputs from RAG using this index to the newer
index from our main results (December 2018). We prepare a list of 82 world leaders who had changed
7

In [66]:
query = "What are the two sub-layers in each encoder layer of the transformer model?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What are the two sub-layers in each encoder layer of the transformer model?


Response:


The two sub-layers in each encoder layer of the transformer model are:

1. **Multi-head self-attention mechanism**: This sub-layer allows the model to focus on different parts of the input sequence simultaneously, capturing various relationships and dependencies between the input tokens.

2. **Position-wise fully connected feed-forward network**: This sub-layer consists of two linear transformations with a ReLU activation in between, applied to each position separately and identically. 

Additionally, each of these sub-layers is enhanced with a residual connection followed by layer normalization to improve model performance.

Sources:
Metadata: {'id': '5d6c0989-c03e-4071-9091-82b17ff44825', 'page': 2, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf', 'title': '1706.03762v7.pdf'}
Content Brief:


...

Focuses on the Transformer model architecture, which relies entirely on self-attention mechanisms for computing input and output representations, eliminating the need for recurrent neural networks (RNNs) or convolutions. It outlines the encoder-decoder structure commonly used in competitive neural sequence transduction models, where the encoder processes input sequences into continuous representations, and the decoder generates output sequences in an auto-regressive manner.
entirely on self-attention to compute representations of its input and output without using sequence-
aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate
self-attention and discuss its advantages over models such as [17, 18] and [9].
3
Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 35].
Here, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence
of continuous representations z = (z1, ..., zn). Given z, the decoder then generates an output
sequence (y1, ..., ym) of symbols one element at a time. At each step the model is auto-regressive
[10], consuming the previously generated symbols as additional input when generating the next.
2


Metadata: {'id': '5cc7fe93-1679-47e8-b0f3-3b47a4b5c4ed', 'page': 9, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf', 'title': '1706.03762v7.pdf'}
Content Brief:


Focuses on the performance of the Transformer model in English constituency parsing, comparing its results to various established parsers, both in discriminative and semi-supervised settings. It highlights the model's competitive F1 scores, demonstrating its effectiveness even without task-specific tuning, and contrasts its performance with that of recurrent neural network models.
Table 4: The Transformer generalizes well to English constituency parsing (Results are on Section 23
of WSJ)
Parser
Training
WSJ 23 F1
Vinyals & Kaiser el al. (2014) [37]
WSJ only, discriminative
88.3
Petrov et al. (2006) [29]
WSJ only, discriminative
90.4
Zhu et al. (2013) [40]
WSJ only, discriminative
90.4
Dyer et al. (2016) [8]
WSJ only, discriminative
91.7
Transformer (4 layers)
WSJ only, discriminative
91.3
Zhu et al. (2013) [40]
semi-supervised
91.3
Huang & Harper (2009) [14]
semi-supervised
91.3
McClosky et al. (2006) [26]
semi-supervised
92.1
Vinyals & Kaiser el al. (2014) [37]
semi-supervised
92.1
Transformer (4 layers)
semi-supervised
92.7
Luong et al. (2015) [23]
multi-task
93.0
Dyer et al. (2016) [8]
generative
93.3
increased the maximum output length to input length + 300. We used a beam size of 21 and α = 0.3
for both WSJ only and the semi-supervised setting.
Our results in Table 4 show that despite the lack of task-specific tuning our model performs sur-
prisingly well, yielding better results than all previously reported models with the exception of the
Recurrent Neural Network Grammar [8].
In contrast to RNN sequence-to-sequence models [37], the Transformer outperforms the Berkeley-
Parser [29] even when training only on the WSJ training set of 40K sentences.
7
Conclusion
In this work, we presented the Transformer, the first sequence transduction model based entirely on
attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with
multi-headed self-attention.
For translation tasks, the Transformer can be trained significantly faster than architectures based
on recurrent or convolutional layers. On both WMT 2014 English-to-German and WMT 2014
English-to-French translation tasks, we achieve a new state of the art. In the former task our best
model outperforms even all previously reported ensembles.
We are excited about the future of attention-based models and plan to apply them to other tasks. We
plan to extend the Transformer to problems involving input and output modalities other than text and
to investigate local, restricted attention mechanisms to efficiently handle large inputs and outputs
such as images, audio and video. Making generation less sequential is another research goals of ours.
The code we used to train and evaluate our models is available at https://github.com/
tensorflow/tensor2tensor.
Acknowledgements
We are grateful to Nal Kalchbrenner and Stephan Gouws for their fruitful
comments, corrections and inspiration.
References
[1] Jimmy Lei Ba, Jamie Ryan Kiros, and Geoffrey E Hinton. Layer normalization. arXiv preprint
arXiv:1607.06450, 2016.
[2] Dzmitry Bahdanau, Kyunghyun Cho, and Yoshua Bengio. Neural machine translation by jointly
learning to align and translate. CoRR, abs/1409.0473, 2014.
[3] Denny Britz, Anna Goldie, Minh-Thang Luong, and Quoc V. Le. Massive exploration of neural
machine translation architectures. CoRR, abs/1703.03906, 2017.
[4] Jianpeng Cheng, Li Dong, and Mirella Lapata. Long short-term memory-networks for machine
reading. arXiv preprint arXiv:1601.06733, 2016.
10


Metadata: {'id': '24de9e73-466f-4d77-bccf-5eddc8c9f165', 'page': 1, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf', 'title': '1706.03762v7.pdf'}
Content Brief:


Focuses on the limitations of recurrent neural networks and the advantages of attention mechanisms in sequence modeling, introducing the Transformer architecture as a novel approach that eliminates recurrence to enhance parallelization and improve translation quality.
1
Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements in computational efficiency through factorization tricks [21] and conditional
computation [32], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduc-
tion models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100 GPUs.
2
Background
The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU
[16], ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic building
block, computing hidden representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difficult to learn dependencies between distant positions [12]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-
aligned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the first transduction model relying

In [67]:
query = "Explain how positional encoding is implemented in Transformers and why it is neccessary?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


Explain how positional encoding is implemented in Transformers and why it is neccessary?


Response:


Positional encoding is a crucial component in the Transformer architecture, which is designed to handle sequences without relying on recurrent neural networks (RNNs) or convolutions. Here’s how it is implemented and why it is necessary:

### Implementation of Positional Encoding

1. **Purpose**: Since the Transformer model processes input sequences in parallel, it lacks the inherent sequential nature of RNNs, which means it does not have a built-in mechanism to understand the order of tokens in a sequence. Positional encoding is introduced to provide this information.

2. **Encoding Method**: The positional encoding is typically implemented using sine and cosine functions of different frequencies. Each position in the input sequence is assigned a unique vector that encodes its position. The formula used for the positional encoding is:
   - For each position \( pos \) and dimension \( i \):
     - \( PE(pos, 2i) = \sin(pos / 10000^{2i/d_{model}}) \)
     - \( PE(pos, 2i + 1) = \cos(pos / 10000^{2i/d_{model}}) \)
   - Here, \( d_{model} \) is the dimensionality of the embeddings, and \( i \) ranges from 0 to \( d_{model} - 1 \).

3. **Integration with Input Embeddings**: The positional encodings are added to the input embeddings of the tokens. This addition allows the model to incorporate both the content of the tokens and their positional information, enabling it to learn the relationships between tokens based on their positions in the sequence.

### Necessity of Positional Encoding

1. **Understanding Sequence Order**: The primary reason for using positional encoding is to give the model a sense of the order of tokens in the input sequence. Without this information, the model would treat the input as a bag of words, losing the context that is crucial for understanding language.

2. **Facilitating Attention Mechanisms**: The attention mechanisms in the Transformer rely on the relationships between tokens. By incorporating positional information, the model can better determine how much focus to place on different tokens based on their positions, which is essential for tasks like translation and text generation.

3. **Enhancing Model Performance**: Positional encoding allows the Transformer to capture long-range dependencies and relationships between tokens more effectively than models that do not use such encodings. This leads to improved performance in various sequence transduction tasks.

In summary, positional encoding is implemented through sine and cosine functions to provide unique positional information to each token in the sequence. It is necessary for enabling the Transformer model to understand the order of tokens, facilitating effective attention mechanisms, and enhancing overall model performance in sequence-related tasks.

Sources:
Metadata: {'id': '5d6c0989-c03e-4071-9091-82b17ff44825', 'page': 2, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf', 'title': '1706.03762v7.pdf'}
Content Brief:


...

Focuses on the implementation of multi-head attention in the Transformer model, detailing how multiple attention heads operate in parallel to enhance the model's ability to capture diverse information from different representation subspaces. It also describes the structure of the attention mechanism and its application in both encoder-decoder and self-attention layers, emphasizing the importance of maintaining the auto-regressive property in the decoder.
output values. These are concatenated and once again projected, resulting in the final values, as
depicted in Figure 2.
Multi-head attention allows the model to jointly attend to information from different representation
subspaces at different positions. With a single attention head, averaging inhibits this.
MultiHead(Q, K, V ) = Concat(head1, ..., headh)W O
where headi = Attention(QW Q
i , KW K
i , V W V
i )
Where the projections are parameter matrices W Q
i
∈Rdmodel×dk, W K
i
∈Rdmodel×dk, W V
i
∈Rdmodel×dv
and W O ∈Rhdv×dmodel.
In this work we employ h = 8 parallel attention layers, or heads. For each of these we use
dk = dv = dmodel/h = 64. Due to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full dimensionality.
3.2.3
Applications of Attention in our Model
The Transformer uses multi-head attention in three different ways:
• In "encoder-decoder attention" layers, the queries come from the previous decoder layer,
and the memory keys and values come from the output of the encoder. This allows every
position in the decoder to attend over all positions in the input sequence. This mimics the
typical encoder-decoder attention mechanisms in sequence-to-sequence models such as
[38, 2, 9].
• The encoder contains self-attention layers. In a self-attention layer all of the keys, values
and queries come from the same place, in this case, the output of the previous layer in the
encoder. Each position in the encoder can attend to all positions in the previous layer of the
encoder.
• Similarly, self-attention layers in the decoder allow each position in the decoder to attend to
all positions in the decoder up to and including that position. We need to prevent leftward
information flow in the decoder to preserve the auto-regressive property. We implement this
inside of scaled dot-product attention by masking out (setting to −∞) all values in the input
of the softmax which correspond to illegal connections. See Figure 2.
3.3
Position-wise Feed-Forward Networks
In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully
connected feed-forward network, which is applied to each position separately and identically. This
consists of two linear transformations with a ReLU activation in between.
FFN(x) = max(0, xW1 + b1)W2 + b2
(2)
While the linear transformations are the same across different positions, they use different parameters
from layer to layer. Another way of describing this is as two convolutions with kernel size 1.
The dimensionality of input and output is dmodel = 512, and the inner-layer has dimensionality
dff = 2048.
3.4
Embeddings and Softmax
Similarly to other sequence transduction models, we use learned embeddings to convert the input
tokens and output tokens to vectors of dimension dmodel. We also use the usual learned linear transfor-
mation and softmax function to convert the decoder output to predicted next-token probabilities. In
our model, we share the same weight matrix between the two embedding layers and the pre-softmax
linear transformation, similar to [30]. In the embedding layers, we multiply those weights by √dmodel.
5


Metadata: {'id': '24de9e73-466f-4d77-bccf-5eddc8c9f165', 'page': 1, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf', 'title': '1706.03762v7.pdf'}
Content Brief:


Focuses on the limitations of recurrent neural networks and the advantages of attention mechanisms in sequence modeling, introducing the Transformer architecture as a novel approach that eliminates recurrence to enhance parallelization and improve translation quality.
1
Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements in computational efficiency through factorization tricks [21] and conditional
computation [32], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduc-
tion models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100 GPUs.
2
Background
The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU
[16], ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic building
block, computing hidden representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difficult to learn dependencies between distant positions [12]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-
aligned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the first transduction model relying


Metadata: {'id': '7b8b997d-cf9b-464c-bb98-0e3414f7f1bc', 'page': 0, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf', 'title': '1706.03762v7.pdf'}
Content Brief:


Focuses on the authorship and permissions related to the research paper "Attention Is All You Need," which introduces the Transformer model for sequence transduction tasks. It highlights the contributions of various authors from Google Brain and Google Research, as well as the paper's presentation at the 31st Conference on Neural Information Processing Systems (NIPS 2017).
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-
to-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.8 after
training for 3.5 days on eight GPUs, a small fraction of the training costs of the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data.
∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started
the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and
has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head
attention and the parameter-free position representation and became the other person involved in nearly every
detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and
tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and
efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and
implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating
our research.
†Work performed while at Google Brain.
‡Work performed while at Google Research.
31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA.
arXiv:1706.03762v7  [cs.CL]  2 Aug 2023

In [68]:
query = "Describe the concept of multi-head Attention in the transformer architecture?how it is beneficial?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


Describe the concept of multi-head Attention in the transformer architecture?how it is beneficial?


Response:


### Multi-Head Attention in the Transformer Architecture

**Concept Overview:**
Multi-Head Attention is a crucial component of the Transformer architecture, which enhances the model's ability to capture diverse information from different representation subspaces. Instead of performing a single attention function with d_model-dimensional keys, values, and queries, Multi-Head Attention projects the queries, keys, and values multiple times (h times) using different learned linear projections. This allows the model to jointly attend to information from various representation subspaces at different positions.

**Mathematical Formulation:**
The Multi-Head Attention mechanism can be expressed mathematically as follows:

\[
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h) W^O
\]

where each head is computed as:

\[
\text{head}_i = \text{Attention}(Q W^Q_i, K W^K_i, V W^V_i)
\]

Here, \(W^Q_i\), \(W^K_i\), and \(W^V_i\) are parameter matrices for the queries, keys, and values, respectively, and \(W^O\) is the output projection matrix. The projections reduce the dimensionality of each head, allowing for efficient computation while maintaining a similar total computational cost to that of single-head attention with full dimensionality.

**Benefits of Multi-Head Attention:**
1. **Diverse Representation Capture:** By using multiple attention heads, the model can focus on different parts of the input sequence simultaneously, capturing various aspects of the data that a single attention head might miss. This is particularly beneficial for understanding complex relationships in the data.

2. **Parallel Processing:** Multi-Head Attention allows for parallel computation of attention heads, which enhances the efficiency of the model. Each head processes its own set of projections independently, leading to faster training and inference times.

3. **Improved Contextual Understanding:** The ability to attend to different representation subspaces enables the model to better understand the context and relationships between words in a sequence, which is essential for tasks like translation and summarization.

4. **Maintaining Auto-Regressive Properties:** In the decoder, Multi-Head Attention is designed to prevent positions from attending to subsequent positions, preserving the auto-regressive property necessary for generating sequences in a controlled manner.

In summary, Multi-Head Attention is a powerful mechanism that significantly enhances the capabilities of the Transformer model by allowing it to process information in a more nuanced and efficient way.

Sources:
Metadata: {'id': '054731fe-1b71-4549-9604-c58c028ccdf5', 'page': 3, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf', 'title': '1706.03762v7.pdf'}
Content Brief:


...

Focuses on the limitations of recurrent neural networks and the advantages of attention mechanisms in sequence modeling, introducing the Transformer architecture as a novel approach that eliminates recurrence to enhance parallelization and improve translation quality.
1
Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [38, 24, 15].
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements in computational efficiency through factorization tricks [21] and conditional
computation [32], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduc-
tion models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100 GPUs.
2
Background
The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU
[16], ByteNet [18] and ConvS2S [9], all of which use convolutional neural networks as basic building
block, computing hidden representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difficult to learn dependencies between distant positions [12]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-
aligned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [34].
To the best of our knowledge, however, the Transformer is the first transduction model relying


Metadata: {'id': '5d6c0989-c03e-4071-9091-82b17ff44825', 'page': 2, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf', 'title': '1706.03762v7.pdf'}
Content Brief:


Focuses on the architecture of the Transformer model, detailing the structure of the encoder and decoder stacks, which consist of multiple layers incorporating self-attention mechanisms and feed-forward networks. It describes the use of residual connections and layer normalization to enhance model performance and outlines the specific configurations for both the encoder and decoder components.
Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1
Encoder and Decoder Stacks
Encoder:
The encoder is composed of a stack of N = 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
wise fully connected feed-forward network. We employ a residual connection [11] around each of
the two sub-layers, followed by layer normalization [1]. That is, the output of each sub-layer is
LayerNorm(x + Sublayer(x)), where Sublayer(x) is the function implemented by the sub-layer
itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimension dmodel = 512.
Decoder:
The decoder is also composed of a stack of N = 6 identical layers. In addition to the two
sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head
attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions less than i.
3.2
Attention
An attention function can be described as mapping a query and a set of key-value pairs to an output,
where the query, keys, values, and output are all vectors. The output is computed as a weighted sum
3


Metadata: {'id': '0e0a3d06-2d94-44da-92b9-193086cbfd09', 'page': 12, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/1706.03762v7.pdf', 'title': '1706.03762v7.pdf'}
Content Brief:


Focuses on visualizing the attention mechanism in the Transformer model, specifically illustrating how attention heads track long-distance dependencies within sentences. The example provided highlights the attention behavior related to the verb "making" in a given sentence, demonstrating the model's ability to capture contextual relationships.
Attention Visualizations
It
is
in
this
spirit
that
a
majority
of
American
governments
have
passed
new
laws
since
2009
making
the
registration
or
voting
process
more
difficult
.
<EOS>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
It
is
in
this
spirit
that
a
majority
of
American
governments
have
passed
new
laws
since
2009
making
the
registration
or
voting
process
more
difficult
.
<EOS>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
Figure 3: An example of the attention mechanism following long-distance dependencies in the
encoder self-attention in layer 5 of 6. Many of the attention heads attend to a distant dependency of
the verb ‘making’, completing the phrase ‘making...more difficult’. Attentions here shown only for
the word ‘making’. Different colors represent different heads. Best viewed in color.
13

In [69]:
query = "What is few-shot learning, how does GPT-3 implement it during inference?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What is few-shot learning, how does GPT-3 implement it during inference?


Response:


**Few-Shot Learning and GPT-3 Implementation During Inference**

**Definition of Few-Shot Learning:**
Few-shot learning refers to the ability of a model to learn and perform tasks with only a small number of examples or demonstrations. In the context of natural language processing (NLP), this means that a model can understand and execute tasks based on just a few provided instances, without the need for extensive training or fine-tuning on a specific dataset.

**GPT-3's Implementation of Few-Shot Learning:**
GPT-3, a large-scale autoregressive language model with 175 billion parameters, implements few-shot learning during inference in the following ways:

1. **In-Context Learning:**
   - GPT-3 utilizes in-context learning, where it processes sequences of text that include task instructions and examples. This allows the model to adapt to various tasks based on the context provided in the input.
   - During inference, GPT-3 can take in multiple demonstrations (typically between 10 to 100) as part of its context window, which helps it understand the task requirements better.

2. **Performance Across Settings:**
   - The model is evaluated under three conditions: zero-shot (no examples), one-shot (one example), and few-shot (multiple examples). The performance improves significantly as the number of examples increases, demonstrating the model's ability to learn from context.
   - For instance, GPT-3 achieved an F1 score of 85.0 in the few-shot setting on the CoQA dataset, compared to 81.5 in zero-shot and 84.0 in one-shot settings.

3. **Task Adaptation:**
   - GPT-3 shows proficiency in tasks that require rapid adaptation or reasoning, such as unscrambling words, performing arithmetic, and using novel words in sentences after seeing them defined only once.
   - The model can generate coherent outputs, such as synthetic news articles, that are often indistinguishable from human-written content, showcasing its few-shot capabilities.

4. **Impact of Model Size:**
   - The performance in few-shot learning settings improves dramatically with the size of the model. Larger models like GPT-3 are more proficient at in-context learning, as evidenced by its performance metrics across various NLP tasks.

5. **Limitations:**
   - Despite its strengths, GPT-3 also faces challenges in certain tasks, particularly in natural language inference and some reading comprehension datasets, where its few-shot performance may not be competitive with state-of-the-art fine-tuned models.

In summary, GPT-3's implementation of few-shot learning during inference allows it to effectively adapt to new tasks using minimal examples, leveraging its large model size and in-context learning capabilities to achieve strong performance across a variety of NLP benchmarks.

Sources:
Metadata: {'id': 'aa936614-b2e6-4d07-84c7-ea68c00b9dbc', 'page': 4, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.14165v4.pdf', 'title': '2005.14165v4.pdf'}
Content Brief:


...

Focuses on the performance metrics of GPT-3 on various cloze and completion tasks, specifically highlighting its improvements over state-of-the-art results on the LAMBADA dataset and its capabilities in few-shot learning settings. It discusses the model's ability to infer task requirements from examples and the impact of model size on performance.
Setting
LAMBADA
(acc)
LAMBADA
(ppl)
StoryCloze
(acc)
HellaSwag
(acc)
SOTA
68.0a
8.63b
91.8c
85.6d
GPT-3 Zero-Shot
76.2
3.00
83.2
78.9
GPT-3 One-Shot
72.5
3.35
84.7
78.1
GPT-3 Few-Shot
86.4
1.92
87.7
79.3
Table 3.2: Performance on cloze and completion tasks. GPT-3 signiﬁcantly improves SOTA on LAMBADA while
achieving respectable performance on two difﬁcult completion prediction datasets. a[Tur20] b[RWC+19] c[LDL19]
d[LCH+20]
Figure 3.2: On LAMBADA, the few-shot capability of language models results in a strong boost to accuracy. GPT-3
2.7B outperforms the SOTA 17B parameter Turing-NLG [Tur20] in this setting, and GPT-3 175B advances the state of
the art by 18%. Note zero-shot uses a different format from one-shot and few-shot as described in the text.
and [Tur20]) and argue that “continuing to expand hardware and data sizes by orders of magnitude is not the path
forward”. We ﬁnd that path is still promising and in a zero-shot setting GPT-3 achieves 76% on LAMBADA, a gain of
8% over the previous state of the art.
LAMBADA is also a demonstration of the ﬂexibility of few-shot learning as it provides a way to address a problem that
classically occurs with this dataset. Although the completion in LAMBADA is always the last word in a sentence, a
standard language model has no way of knowing this detail. It thus assigns probability not only to the correct ending but
also to other valid continuations of the paragraph. This problem has been partially addressed in the past with stop-word
ﬁlters [RWC+19] (which ban “continuation” words). The few-shot setting instead allows us to “frame” the task as a
cloze-test and allows the language model to infer from examples that a completion of exactly one word is desired. We
use the following ﬁll-in-the-blank format:
Alice was friends with Bob. Alice went to visit her friend
. →Bob
George bought some baseball equipment, a ball, a glove, and a
. →
When presented with examples formatted this way, GPT-3 achieves 86.4% accuracy in the few-shot setting, an increase
of over 18% from the previous state-of-the-art. We observe that few-shot performance improves strongly with model
size. While this setting decreases the performance of the smallest model by almost 20%, for GPT-3 it improves accuracy
by 10%. Finally, the ﬁll-in-blank method is not effective one-shot, where it always performs worse than the zero-shot
setting. Perhaps this is because all models still require several examples to recognize the pattern.
12


Metadata: {'id': 'e09c8c5e-fb83-4b45-a01b-cc9e5b318c49', 'page': 2, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.14165v4.pdf', 'title': '2005.14165v4.pdf'}
Content Brief:


Focuses on the concept of in-context learning within the framework of the GPT-3 model, illustrating how the model processes sequences during inference. It highlights that the sequences presented in the accompanying diagram are not representative of the training data but demonstrate the presence of repeated sub-tasks within a single sequence. This explanation is part of a broader discussion on the model's architecture and its ability to adapt to various tasks without fine-tuning.
the forward-pass upon each sequence. The sequences in this diagram are not intended to be representative of the data a
model would see during pre-training, but are intended to show that there are sometimes repeated sub-tasks embedded
within a single sequence.
3


Metadata: {'id': '782ec2e9-cb57-4ee8-8cc4-ecef0fffa64a', 'page': 0, 'source': '/content/drive/My Drive/Colab_Notebooks/RAG_Project_Dataset/2005.14165v4.pdf', 'title': '2005.14165v4.pdf'}
Content Brief:


Focuses on the introduction and abstract of the research paper, highlighting the advancements in natural language processing (NLP) through the development of GPT-3, a large-scale language model. It discusses the model's ability to perform tasks in a few-shot learning setting without the need for extensive fine-tuning, and outlines its performance across various NLP benchmarks, as well as the societal implications of its capabilities.
Language Models are Few-Shot Learners
Tom B. Brown∗
Benjamin Mann∗
Nick Ryder∗
Melanie Subbiah∗
Jared Kaplan†
Prafulla Dhariwal
Arvind Neelakantan
Pranav Shyam
Girish Sastry
Amanda Askell
Sandhini Agarwal
Ariel Herbert-Voss
Gretchen Krueger
Tom Henighan
Rewon Child
Aditya Ramesh
Daniel M. Ziegler
Jeffrey Wu
Clemens Winter
Christopher Hesse
Mark Chen
Eric Sigler
Mateusz Litwin
Scott Gray
Benjamin Chess
Jack Clark
Christopher Berner
Sam McCandlish
Alec Radford
Ilya Sutskever
Dario Amodei
OpenAI
Abstract
Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training
on a large corpus of text followed by ﬁne-tuning on a speciﬁc task. While typically task-agnostic
in architecture, this method still requires task-speciﬁc ﬁne-tuning datasets of thousands or tens of
thousands of examples. By contrast, humans can generally perform a new language task from only
a few examples or from simple instructions – something which current NLP systems still largely
struggle to do. Here we show that scaling up language models greatly improves task-agnostic,
few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art ﬁne-
tuning approaches. Speciﬁcally, we train GPT-3, an autoregressive language model with 175 billion
parameters, 10x more than any previous non-sparse language model, and test its performance in
the few-shot setting. For all tasks, GPT-3 is applied without any gradient updates or ﬁne-tuning,
with tasks and few-shot demonstrations speciﬁed purely via text interaction with the model. GPT-3
achieves strong performance on many NLP datasets, including translation, question-answering, and
cloze tasks, as well as several tasks that require on-the-ﬂy reasoning or domain adaptation, such as
unscrambling words, using a novel word in a sentence, or performing 3-digit arithmetic. At the same
time, we also identify some datasets where GPT-3’s few-shot learning still struggles, as well as some
datasets where GPT-3 faces methodological issues related to training on large web corpora. Finally,
we ﬁnd that GPT-3 can generate samples of news articles which human evaluators have difﬁculty
distinguishing from articles written by humans. We discuss broader societal impacts of this ﬁnding
and of GPT-3 in general.
∗Equal contribution
†Johns Hopkins University, OpenAI
Author contributions listed at end of paper.
arXiv:2005.14165v4  [cs.CL]  22 Jul 2020